In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

#Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# dx = 1 km; Np = 1M; Nt = 5 min
data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_1e6.nc', decode_timedelta=True) #***
parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_1e6.nc', decode_timedelta=True) #***
res='1km'
Np_str='1e6'

# # dx = 1 km; Np = 1M; Nt = 1 min
# data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_1e6_1min.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_1e6_1min.nc', decode_timedelta=True) #***
# res='1km'
# Np_str='1e6'


# dx = 250 m
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_250m.nc', decode_timedelta=True) #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_250m.nc', decode_timedelta=True) #***

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
def get_time(data,t,init):
    (init_day,init_hour,init_min)=init[0],init[1],init[2]
    times=data['time'].values/(1e9 * 60); time_inc=times.astype(int)[1]-times.astype(int)[0]
    current_min=init_hour*60+init_min+time_inc*t;
    
    days = init_day + (current_min // (24 * 60))
    
    remain_min = (init_min+time_inc*t) % (24 * 60); 
    hours = (init_hour + (remain_min // 60)) % 24
    mins = remain_min % 60

    ##############################################
    days=str(days);hours=str(hours);mins=str(mins)
    if len(days)==1:days='0'+days
    if len(hours)==1:hours='0'+hours
    if len(mins)==1:mins='0'+mins
    ##############################################

    combo=days+":"+hours+":"+mins
    return(days,hours,mins),(combo)

In [ ]:
################################################################################################

In [ ]:
def get_data(varname,t_slice,x_slice):
    out = data[varname].isel(time=t_slice,xh=x_slice).data
    return out

In [ ]:
t1=72
t2=102
t_slice=slice(t1,t2+1)
x_slice=slice(150,250)



In [ ]:
#ESTIMATING NP
Lx=100e3
Ly=200e3
# dm=2e6
dm=5e5

out=((Lx*Ly)*1e4)/dm
print(f"{out:.0e}")

In [ ]:
varname='qc'
var=get_data(varname,t_slice,x_slice)
varname='qi'
var+=get_data(varname,t_slice,x_slice)

In [ ]:
#Horizontal 
t=15; timestamp=get_time(data,t1+t,(0,6,0))[1]
z=10;print(f"{data['zh'][z].item():.2f} km")

var_z=var[t,z]
plt.contourf(var_z)
plt.title(f"qcqi at t = {timestamp}")
plt.ylabel('y (km)'); plt.xlabel('x (km)') #not full domain

In [ ]:
#Vertical
t=15; timestamp=get_time(data,t1+t,(0,6,0))[1]
x=7

var_x=var[t,:,:,x]
plt.contourf(var_x)
plt.title(f"qcqi at t = {timestamp}")
plt.ylabel('z (km)'); plt.xlabel('y (km)') #not full domain

ax = plt.gca()
fix_tick_labels([ax],data,'z','y',10,2,'center',2,False)

In [ ]:
#Vertical
t=15; timestamp=get_time(data,t1+t,(0,6,0))[1]
y=62

var_y=var[t,:,y]
plt.contourf(var_y)
plt.title(f"qcqi at t = {timestamp}")
plt.ylabel('z (km)'); plt.xlabel('x(km)') #not full domain

ax = plt.gca()
fix_tick_labels([ax],data,'z','y',10,2,'center',2,False)